In [1]:
from dash import Dash, html, dcc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template

import plotly.express as px 
import plotly.graph_objects as go
import plotly.io as pio

import pandas as pd 
import numpy as np 
import seaborn as sns 

#  Set default global styles for graphics
pio.templates.default = "plotly"
pio.templates["plotly"] = pio.templates["plotly"].update({
    "layout": {
        "paper_bgcolor": "white",  # Paper background color
        "plot_bgcolor": "#3E5B76",    # Graphic background color
        "font": {"color": "#7B3F00"},    # Text color
    }
})


candy =pd.read_csv("candy-data.csv")

for column in ["sugarpercent", "pricepercent", "winpercent"]:
    if column == "winpercent":
        candy[column] = candy[column] / 100
    candy[column[:-7] + "percentilerange"] = np.select( 
        condlist= [
            candy[column] <= 0.25,
            candy[column] <= 0.50,
            candy[column] <= 0.75
        ],
        choicelist= ['Low (0-25%)', 'Medium-Low (25%-50%)', 'Medium-High (50%-75%)'],
        default='High (75%-100%)'
    )
    
candy["winpercent"] = candy["winpercent"] * 100  # change back



dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css" 

app = Dash(__name__, 
           external_stylesheets=[dbc.themes.BOOTSTRAP, dbc_css ],
           meta_tags= [{"name": "viewport", 
               "content": "width=device-width, initial-scale=1"}]
          )

# load_figure_template("LITERA")


     

app.layout = dbc.Container(
    style= {"backgroundColor": "#7B3F00"},
    children = [
    dbc.Card(dbc.Col([
        html.H1("Halloween Candy Rankings: Analysis of 85 Popular Candies", 
                style={"text-align":"center", "fontSize":36}),
        html.Br(),
    ])),
    
    html.Br(),
    dcc.Tabs([
        dcc.Tab(
            label = "Composition", children =  [
                html.Br(),
                dbc.Card([
                    dbc.Row([
                        dbc.Col(dcc.Graph(id="top-10-win"), width={"size": 8, "offset":1}),
                        dbc.Col([
                            dcc.Dropdown(id="metric-choice",
                                         options = [{"value" : "pricepercent", "label":"Price Percent"}, 
                                                     {"value":"sugarpercent", "label":"Sugar Percent"}, 
                                                     {"value": "winpercent", "label":"Win Percent"}],
                                         value="winpercent"
                                        )], width=2)
                    ]),
                ]),
                
                html.Br(),
                dbc.Card([
                     dbc.Row([
                         dbc.Col(dcc.Graph(id="packeging-pie"), width=5),
                         dbc.Col([
                             dbc.Row([
                                 dbc.Col( dcc.Graph(id="percent_classification_bar")),
                                 dbc.Col(dcc.Dropdown(id="percent_dropdown",
                                          options = [
                                              {"value": "sugarpercentilerange", "label":"Sugar Percent Range"},
                                              {"value": "pricepercentilerange", "label":"Price Percent Range"},
                                              {"value": "winpercentilerange", "label":"Wind Percent Range"}],
                                          value = "pricepercentilerange"), width=4)
                             ])
                             
                         ], width=7)
                     ])
                    ]),
                html.Br(),
                dbc.Card([
                    dbc.Row([
                        dbc.Col(dcc.Graph(id="candy_flavor_bar")),
                        dbc.Col(dcc.Graph(id="texture_pie")),
               ]),
                ])
            ]),
        
        dcc.Tab(label = "Correlation Analysis", children= [
            html.Br(),
            dbc.Col(
                dbc.Row([
                    dbc.Col(dcc.Graph(id="correlation-fig")),
                    dbc.Col([
                        dcc.Markdown("Choose X Axis :"),
                        dcc.Dropdown(
                            id="xaxis-scatter",
                            options = [
                                {"value" : "pricepercent", "label":"Price Percent"}, 
                                {"value":"sugarpercent", "label":"Sugar Percent"},
                                {"value": "winpercent", "label":"Win Percent"}],
                            value="winpercent"),
                        html.Br(),
                        dcc.Markdown("Choose Y Axis :"),
                        dcc.Dropdown(
                            id="yaxis-scatter",
                            options = [
                                {"value" : "pricepercent", "label":"Price Percent"}, 
                                {"value":"sugarpercent", "label":"Sugar Percent"},
                                {"value": "winpercent", "label":"Win Percent"}],
                            value="winpercent")
                    ], width=2)
            ])),
            html.Br(),
            dbc.Card(dbc.Col(dcc.Graph(id="correlation-matrix")))
               
        ])
    ], className="dbc")
    
])


@app.callback(
    Output(component_id="top-10-win", component_property="figure"),
    Output(component_id="packeging-pie", component_property="figure"),
    Output(component_id="percent_classification_bar", component_property="figure"),
    Output(component_id="candy_flavor_bar", component_property="figure"),
    Output(component_id="texture_pie", component_property="figure"),
    Input(component_id="metric-choice", component_property="value"),
    Input(component_id="percent_dropdown", component_property="value")
)
def composition_graphs(metric, percent_range):
    if not metric and not percent_range:
        raise PreventUpdate
    
    top10_win = (candy.loc[:, ["competitorname", f"{metric}"]]
                   .sort_values(by=f"{metric}", ascending=False)
                   .iloc[:10].sort_values(by=f"{metric}", ascending=True)
                   .round(2))
    
    fig_top10 =  px.bar(data_frame= top10_win,
                        y="competitorname",
                        x=metric,
                        text=metric,
                        title= f"Candy Competitor Name By {metric.title()}"
                       ) 
    
    fig_packaging = px.pie(candy.pluribus.value_counts(), 
       names=candy.pluribus.value_counts().index.map({1:"Multi-Pack", 0:"individual"}), 
       values="count", hole=.7,
       title= "Candy Packaging: Individual vs. Multi-Pack"
      )
    
    percent_df = pd.DataFrame(candy[percent_range].value_counts(ascending=False))  # for dropdown on percentile classification
    percent_class_fig = px.bar(percent_df, 
                               x=percent_df.index, 
                               y="count", 
                               text="count", 
                               title="Number of competitors By wind percent range"
                              )
    
    flabor_candy =  {
        "chocolate" :  candy["chocolate"].sum(),
        "fruity" : candy["fruity"].sum(),
        "caramel" :  candy["caramel"].sum(),
        "peanutyalmondy" :  candy["peanutyalmondy"].sum(),
        "nougat": candy["nougat"].sum()
    }
    
    df_flavor  =  pd.DataFrame(flabor_candy.items(), columns=["flavor", "count"])
    
    fig_flavor_dist =  px.bar(df_flavor.sort_values(by="count", ascending=False), 
                              x="flavor", 
                              y="count", 
                              text="count",
                              title="Distribution of Candy Flavors" 
                              )
    
    fig_texture =  px.pie(candy["crispedricewafer"].value_counts(),   # candy Texture
                          names=candy["crispedricewafer"].value_counts().index.map({1:"Yes", 0:"No"}),
                          values=candy["crispedricewafer"].value_counts(),
                          title= "Does it contain crisped rice, wafers, or a cookie component?",
                          hole= 0.7
                         )
    
    return fig_top10, fig_packaging, percent_class_fig, fig_flavor_dist, fig_texture

    


@app.callback(
    Output(component_id="correlation-fig", component_property="figure"),
    Output(component_id="correlation-matrix", component_property="figure"),
    Input(component_id="xaxis-scatter", component_property="value"),
    Input(component_id="yaxis-scatter", component_property="value")
)
def correlation_graphs(xaxis, yaxis):
    
    # scatter plot between variable
    fig_scatter = px.scatter(candy, 
                             x=xaxis, 
                             y=yaxis, 
                             hover_name= "competitorname", 
                             size="sugarpercent",
                             size_max=50,
                             title= f"Relation Between {xaxis} and {yaxis}"
                            )
    
    # Correlation Matrix with Plotly using the oriented Object approch  : plotly.graph_objects as go
    # Compute the correlation matrix
    corr = candy.select_dtypes(include="number").corr()

    # Create a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # Apply the mask to the correlation matrix
    masked_corr = corr.mask(mask)

    # Create the heatmap using Plotly
    correlation_matrix_fig = go.Figure(
        data=go.Heatmap(
            z=masked_corr.values,
            x=masked_corr.columns,
            y=masked_corr.columns,
            colorscale='RdYlGn',
            zmin=-1,  # Specify the range to align with correlation values
            zmax=1,
            colorbar=dict(title="Correlation values")
        )
    )

    # Update the layout for better appearance
    correlation_matrix_fig.update_layout(
        title='Correlation Matrix Between Candy Variables',
        xaxis_title='Features',
        yaxis_title='Features',
        xaxis_nticks=len(corr.columns),
        yaxis_nticks=len(corr.columns),
#         width=800,  # Width of the heatmap
#         height=600,  # Height of the heatmap
    )
    
    return fig_scatter, correlation_matrix_fig




if __name__ == "__main__":
    app.run_server(debug=True, port=9000)